In [ ]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 15.5 MB/s 
     |████████████████████████████████| 120 kB 87.4 MB/s 
     |████████████████████████████████| 6.6 MB 83.3 MB/s 
     |████████████████████████████████| 1.3 MB 69.1 MB/s 


In [ ]:
pip install git+https://github.com/csebuetnlp/normalizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-y4vswkhv
  Running command git clone -q https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-y4vswkhv
     |████████████████████████████████| 184 kB 15.4 MB/s 
     |████████████████████████████████| 64 kB 3.5 MB/s 
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6885 sha256=18d8ea0c3ff498ddbad3e6bd710a630682735a0ab58df105cf455ecd17e58755
  Stored in directory: /tmp/pip-ephem-wheel-cache-l8v7_st4/wheels/af/b1/ee/b9e2a2f2dd861976a357b6a6fa105aeedf2254016676f6cf8f
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=09c6e783c45725b4624252a2bfe356bd8c2b4fe8eefd265310530316aa3a97be
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl siz

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer


# calling BanglaT5 model
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base", use_fast=False)

In [ ]:
# Mounting colab with Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# It it recommended to not change the dataset's files' name

# 'base' variable has to store the path to the folder where datasets are uploaded/available
# so set the 'base' variable path to the folder of the where you have uploaded the datasets
base = '/content/drive/MyDrive/BQA'        # sample

# 'checkpoint_dir' variable has to store the path to the folder where checkpoints will be saves
# IT IS RECOMMENDED TO SET checkpoint_dir SAME AS base
# so set the 'checkpoint_dir' variable path to the folder of the where you want the check points to be saved
checkpoint_dir = '/content/drive/MyDrive/BQA'          #sample

# 'train_filename' variable has to store the filename of the Train set
# so set the 'train_filename' variable as the name of the Train set's filename
train_filename = 'Train.json'        #sample

# 'validation_file_name' variable has to store the filename of the Validation set
# so set the 'validation_file_name' variable as the name of the validation set's filename
validation_file_name = 'Validation.json'     #sample

In [ ]:
# Loading the Train.json file for train set of BanglaRQA

import json
import os

f = open(os.path.join(base,train_filename))
  
data_test = json.load(f)

data_test.keys()
  

In [ ]:
data = data_test['data']
len(data)

In [ ]:
data[0]

In [ ]:
context = []
question = []
answer = []

for i in range(len(data)):
    for j in range(len(data[i]['qas'])):
        context.append(normalize(data[i]['context']))
        question.append(normalize(data[i]['qas'][j]['question_text']))
        answer.append(normalize(data[i]['qas'][j]['answers']['answer_text'][0]))

In [ ]:
print(len(context))
print(len(question))
print(len(answer))

11912
11912
11912


In [ ]:
# encoding the inputs and outputs

train_encodings = tokenizer(question, context, max_length=1024, padding="max_length", truncation="only_second", return_attention_mask=True, add_special_tokens=True, return_tensors="pt")
train_answer_encodings = tokenizer(answer, max_length=256, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")

In [ ]:
train_answer_encodings = train_answer_encodings['input_ids']
train_answer_encodings[train_answer_encodings==0] = -100

In [ ]:
train_encodings.update({'train_answer_encodings': train_answer_encodings})

In [ ]:
train_encodings

{'input_ids': tensor([[44476,  9973,  2369,  ...,     0,     0,     0],
        [11977,  1198, 44476,  ...,     0,     0,     0],
        [  259, 72061,  3491,  ...,     0,     0,     0],
        ...,
        [  259,  1413,   264,  ...,     0,     0,     0],
        [28900, 25393, 52837,  ...,     0,     0,     0],
        [28900, 25393, 52837,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'train_answer_encodings': tensor([[140227,   8460,      1,  ...,   -100,   -100,   -100],
        [     1,   -100,   -100,  ...,   -100,   -100,   -100],
        [ 36876,  15050,  16752,  ...,   -100,   -100,   -100],
        ...,
        [ 48301,  14346,  12114,  ...,   -100,   -100,   -100],
        [     1,   -100,   -100,  ...,   -100,   -100,   -100],
        [     1,   -100,   

In [ ]:
# Loading the validation set of BanglaRQA

import json
import os

f_val = open(os.path.join(base,validation_file_name))
  
data_val = json.load(f_val)

data_val.keys()
  

dict_keys(['data'])

In [ ]:
data_val = data_val['data']

In [ ]:
context_val = []
question_val = []
answer_val = []

for i in range(len(data_val)):
    for j in range(len(data_val[i]['qas'])):
        context_val.append(normalize(data_val[i]['context']))
        question_val.append(normalize(data_val[i]['qas'][j]['question_text']))
        answer_val.append(normalize(data_val[i]['qas'][j]['answers']['answer_text'][0]))

In [ ]:
context_val[0]

'আস্যাইরিয়ান ও ব্যাবলিয়ান সাহিত্যে সরগন তার অবনমিত অবস্থান থেকে ক্ষমতায় উত্থান ও মেসোপটেমিয়া অভিযানের জন্য কিংবদন্তি গল্পের মুল উপজীব্যে পরিনত হয়েছিলেন, এরকম আরও কিছু কিছু আংশিক কিংবদন্তী উপাখ্যান ছাড়াও সারগনের খোদ নিজের অনেক লিপি আছে যদিও তার বেশিরভাগ পরবর্তী সংস্করনগুলি থেকে নেওয়া। ল্যুভর জাদুঘরে দুটি সারগনিক বিজয় ফলক এর অংশবিশেষ আছে যা সুসা (যেখানে এগুলি খুব সম্ভবত মেসোপটেমিয়া ত্থেকে দ্বাদশ শতাব্দীতে স্থানান্তরিত করা হয়েছিল) থেকে পুনঃউদ্ধার করা হয়েছিল। দৃশ্যত সরগন সেমেটিক (আক্কাদীয়ান) ভাষার লিপির লিখিত আকারে প্রসার ঘটিয়েছিলেন, তিনি আক্কাদ শহর প্রতিষ্ঠিত করার প্রথমদিকে নিজেকে প্রায়শয়ই আক্কাদীয়ান রাজা হিসিবে প্রচার করতেন, পরে তিনি কোন এক সময় কিস শহর অধিগ্রহণ করে নেন, পরবর্তীতে মেসোপটেমিয়ার বৃহদাংশও দখল করে নেন, এবং ক্রমে ক্রমে "সরগন, আক্কদীয়ান রাজা, ইনান্নার তত্ত্বাবধায়ক, কিস এর রাজা, আনুর স্থলাভিষিক্ত, রাজ্যের[মেসোপটেমিয়া] রাজা, এনলিলের রাজ্যপাল[এনসি]" নামে নিজেকে প্রচার করে ছিলেন। যদিও সুমেরিয়ান রাজাদের তালিকার অনেক অনুলিপিতে সারগনের শাসনকাল ৫৬, ৫৫ বা ৫৪ বছর বল

In [ ]:
from collections import Counter

# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    common_tokens = sum(common_tokens.values())
    
    # if there are no common tokens then f1 = 0
    if common_tokens == 0:
        return 0
    
    prec = 1.0 * common_tokens / len(pred_tokens)
    rec = 1.0 * common_tokens / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)


In [ ]:
import numpy

def get_answer(question,context):
  source_encoding=tokenizer(
    question,
    context,
    max_length=1024,
    padding="max_length",
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt").to(device)
  
  #print(source_encoding)
  
  generated_ids=model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams = 1,
      max_length=256,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping= True,
      use_cache = True,
  )
  
  #print(generated_ids)

  preds=[tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in generated_ids]

  return "".join(preds), generated_ids

In [ ]:
import torch

class BanglaRQADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# build datasets for both our training and validation sets
train_dataset = BanglaRQADataset(train_encodings)

In [22]:
# Training part 
# no of epoch = 15
# opimizer = Adam
# at every epoch EM and F1 scores were calculated on validation set of BanglaRQA
# at every epoch model was also saved


from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=5e-5)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

for epoch in range(5):
    # set model to train mode
    model.train()
    model_name = 'mt5_model_weights_epoch_' + str(epoch) + '.pth'
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        
        

        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['train_answer_encodings'].to(device)
        outputs = model(input_ids=input_ids, 
                          attention_mask=attention_mask, 
                            labels = labels)
        
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update 
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'loss': loss,
        
        }, os.path.join(checkpoint_dir,model_name))
    
    optim.zero_grad()
    model.eval()
    f1_total = 0.0
    em_total = 0.0

    with torch.no_grad():
      l = len(context_val)
      for i in range(l):
          question = {
              "context": context_val[i],
              "question": question_val[i]
              }

          pred, ids = get_answer(question["question"],question["context"])
          f1_total = f1_total + compute_f1(pred, answer_val[i])
          em_total = em_total + compute_exact_match(pred, answer_val[i])

      print("Epoch", epoch)
      print('F1: ', f1_total/l)
      print('EM: ', em_total/l)  

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
  0%|          | 0/11912 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
Epoch 0: 100%|██████████| 11912/11912 [1:15:47<00:00,  2.62it/s, loss=0.289]


Epoch 0
F1:  0.5760519160394056
EM:  0.488544474393531


Epoch 1: 100%|██████████| 11912/11912 [1:15:48<00:00,  2.62it/s, loss=0.153]


Epoch 1
F1:  0.657968048854148
EM:  0.5256064690026954


Epoch 2: 100%|██████████| 11912/11912 [1:15:48<00:00,  2.62it/s, loss=0.137]


...

Epoch 3
F1:  0.7204681839750812
EM:  0.5579514824797843


Epoch 4: 100%|██████████| 11912/11912 [1:15:43<00:00,  2.62it/s, loss=0.0211]


Epoch 4
F1:  0.7189439631307885
EM:  0.555256064690027


Epoch 5:   9%|▉         | 1084/11912 [06:53<1:08:51,  2.62it/s, loss=0.198]


KeyboardInterrupt: ignored